In [3]:
! pip install folium
! pip install geopandas


In [1]:
#cargando las bibliotecas
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString


import branca.colormap as br 

In [2]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [7]:
pwd

'c:\\Users\\Antoni\\Documents\\GitHub\\QLAB_Summer_Python\\assignments\\assignment_4'

In [3]:
# Get data of labor
sex_work = pd.read_stata(r'../../../QLAB_Summer_Python/_data/8_trab_sex_20.dta' )
sex_work[ 'dpt_code' ] = sex_work.cod_ubigeo.str[ :2 ].copy()
sex_work[ 'prov_code' ] = sex_work.cod_ubigeo.str[ :4 ].copy()

# Sex work
women_work = sex_work[ sex_work.sex == 'Mujer' ].copy().reset_index( drop = True )

In [4]:
# get data from lima
women_work[ 'month' ] = pd.to_datetime( women_work.month , format = '%B' ) \
                                    .dt.strftime( '%m' ) \
                                    .astype( int )

In [5]:
# Sort by department and month
women_work.sort_values([ 'dpt_code', 'month'], inplace = True )

# Get the total number of women workers by dpt
dpt_women_work = women_work.groupby( [ 'dpt_code', 'month'], as_index = False )[['empl']] \
                            .sum() \
                            .rename( columns = {'empl' :'women_empl'})

# Sort by dpt code and month
dpt_women_work.sort_values([ 'dpt_code', 'month'], inplace = True )

In [6]:
df2 = dpt_women_work.groupby( ['dpt_code'],as_index = False )[['women_empl']].mean()

In [7]:
#SHAPEFILE
dpt_shp = gpd.read_file(r'../../../QLAB_Summer_Python/_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp')

In [8]:
df3 = dpt_shp.merge( df2, left_on = 'CCDD', right_on = 'dpt_code'  )

In [4]:
#Map 1
#First we put Lima as the center of the map,using lat and long
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5
#Then we use this information to locate Peru
map1 = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)
#Finally we specify the geographical separation by department and the information that will be used to distinguish according to colors.
#Because there is no color scheme in follium that goes from white to red, we use the one that goes from yellow to red
fm.Choropleth( 
    geo_data=df3,
    data=df3,
    columns=["dpt_code","women_empl"],
    key_on="feature.properties.dpt_code",
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.8,
    smooth_factor=0.2,
    legend_name='Women employers',
    Highlight= True,
    line_color = "#000000",
    overlay=True,
    nan_fill_color = "White"
    ).add_to(map1)
map1

## MAP 1



NameError: name 'fm' is not defined

In [3]:
##Map2
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5
map2 = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)
#In this case we have to create bins to separate the colors by ranges
bins = [675.33, 20000, 40000, 60000, 100000, 656477.25]  
#Finally we specify the geographical separation by department and the information that will be used to distinguish according to colors.
fm.Choropleth(
    geo_data=df3,
    data=df3,
    columns=["dpt_code", "women_empl"],
    key_on="feature.properties.dpt_code",
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.8,
    smooth_factor=0.2,
    legend_name='Women employers',
    line_color="#000000",
    bins=bins,  # Specify the bins
    nan_fill_color="White",
    highlight=True,
    overlay=True,
    threshold_scale=bins,  # We adjust the range
).add_to(map2)

map2

NameError: name 'fm' is not defined

In [5]:
#mapa 3
#mapa 3
df3_1=df3[ df3.CCDD != "15" ]
#seteamos el palacio de gobierno
lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5
map3 = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)
#In this case we have to create bins to separate the colors by ranges
bins = [675.33, 20000, 40000, 60000, 100000, 656477.25]  
#Finally we specify the geographical separation by department and the information that will be used to distinguish according to colors.
fm.Choropleth(
    geo_data=df3_1,
    data=df3_1,
    columns=["dpt_code", "women_empl"],
    key_on="feature.properties.dpt_code",
    fill_color='YlOrRd',
    fill_opacity=0.8,
    line_opacity=0.8,
    smooth_factor=0.2,
    legend_name='Women employers',
    line_color="#000000",
    bins=bins,  # Specify the bins
    nan_fill_color="White",
    highlight=True,
    overlay=True,
    threshold_scale=bins,  # We adjust the range
).add_to(map3)

map3


NameError: name 'df3' is not defined

In [ ]:
#mapa 4
#generamos la nueva variable en un data frame nuevo:
df4 = sex_work.groupby( ['dpt_code', 'month', 'sex'], as_index = False )[['empl']].sum() \
        .pivot( index = [ 'dpt_code', 'month' ] , columns = 'sex',values='empl') \
        .reset_index()

df4[ 'prop_wom' ] = ( df4.Mujer * 100 / df4.Hombre )
df5 = dpt_shp.merge( df4, left_on = 'CCDD', right_on = 'dpt_code'  )

# MAP 4

La creación del mapa 4 vuelve al archivo muy pesado. Por eso la siguiente parte del codigo está comentado. Quitar el comentario y correr en el propio ordenador para poder observar el mapa y se creará un archivo html con el mapa también.


In [11]:
""" # Lista de meses
meses = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
# Coordenadas para centrar el mapa (puedes ajustarlas según tus necesidades)
lat_palacio = -12.0757538
long_palacio = -76.9863174
#zoom start
zoom_start = 5
# Crear el mapa inicial
map3 = fm.Map(location=[lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start=zoom_start)


# Iterar sobre cada mes
for a in meses:
    # Seleccionar el DataFrame correspondiente al mes
    df_month = df5.loc[df5['month']==a]
    
    # Crear la capa Choropleth para el mes actual
    fm.Choropleth(
        geo_data=df_month,
        data=df_month,
        columns= ["dpt_code", "prop_wom"],
        key_on= "feature.properties.dpt_code",
        fill_opacity= 0.8,
        line_opacity= 0.8,
        smooth_factor= 0.2,
        line_color= "#000000",
        overlay= True,
        nan_fill_color= "White",
        fill_color="YlOrRd",
        legend_name= 'Women employers',
        name=a,
        Highlight=True,
        show=False,
    ).add_to(map3)

# Añadir el control de capas
fm.LayerControl(collapsed=False, autoZIndex=False).add_to(map3)

# Mostrar el mapa
map3

map3.save("mapa4.html") """

' # Lista de meses\nmeses = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]\n# Coordenadas para centrar el mapa (puedes ajustarlas según tus necesidades)\nlat_palacio = -12.0757538\nlong_palacio = -76.9863174\n#zoom start\nzoom_start = 5\n# Crear el mapa inicial\nmap3 = fm.Map(location=[lat_palacio, long_palacio], tiles=\'cartodbpositron\', zoom_start=zoom_start)\n\n\n# Iterar sobre cada mes\nfor a in meses:\n    # Seleccionar el DataFrame correspondiente al mes\n    df_month = df5.loc[df5[\'month\']==a]\n    \n    # Crear la capa Choropleth para el mes actual\n    fm.Choropleth(\n        geo_data=df_month,\n        data=df_month,\n        columns= ["dpt_code", "prop_wom"],\n        key_on= "feature.properties.dpt_code",\n        fill_opacity= 0.8,\n        line_opacity= 0.8,\n        smooth_factor= 0.2,\n        line_color= "#000000",\n        overlay= True,\n        nan_fill_color= "White",\n        fill_color

In [ ]:
#mapa 5

In [ ]:
#mapa 6

In [ ]:
#mapa 7

In [ ]:
#mapa 8